This is the first in (hopefully) a series of blog posts on solutions to weekly programming assignments for [Professor Pascal Van Hentenryck's Discrete Optimization course on Coursera](https://www.coursera.org/learn/discrete-optimization/home/welcome). In my experience so far, the lectures are very good, and the programming assignments are even better. 

Each week, Professor Van Hentenryck introduces a set of algorithmic approaches to combinatorial optimization problems. Then, the weekly programming assignment is an NP-hard optimization problem, along with a set of several different input data files. The different data files are all valid inputs to the same problem, but of different sizes (i.e. it's much harder to find an optimal solution for a given NP-hard problem on a large dataset than a small one)

It's always going to be pretty easy to get a pretty good solution for the smaller data files. To get an optimal solution to the smaller data files, you might have to be a bit more clever. To get a good solution to the larger data files, it's gonna be a bit more work. And to do really well, you need to find optimal (or at least close to optimal) solutions for even the largest data files. 

## The First Week's Programming Assignment

The first week's programming assingment is the [Knapsack Problem](https://en.wikipedia.org/wiki/Knapsack_problem). 

### Problem Formulation
Formally, the problem is defined as:

maximize:  $\sum_{i \in 0...n-1} v_i x_i$

subject to: $\sum_{i in 0...n-1} w_i x_i \leq K   $, $x \in {0,1}$

If this looks confusing, here's the plain english interpretation of the problem:
- We are presented with a set of items, and a knapsack
- Each item has a value $(v)$ and weight $(w)$. Our knapsack can hold a maximum combined weight of $K$
- $x_i$ is 0 or 1 depending on if we place item $i$ in our knapsack

Hopefully now the math above looks pretty straightforward: find the set of items that maximizes their combined value, such that the combined weights of the selected items don't exceed our knapsack weight capacity.

### Data Format Specification
Each input file we're given has n+1 lines in the following format:

```
n K 
v_0 w_0
v_1 w_1
...
v_n-1 w_n-1
```

...where the $n$ and $K$ on the top line represent the number of items we have to select from, and the capacity of the knapsack. Each line after the first describes an item; $v_i$ & $ w_i$

For our solution, we'll output two lines of the format:

```
obj opt 
x_0 x_1 ... x_n-1
```
...where:
- 'obj' will be the total value of our selected items, 
- 'opt' is 1 if we know our solution is optimal, and 0 otherwise
- x_i... are all 0 or 1 for whether we selected that item or not  

## Jumping into the code
As a jumping off point, [we're given the following naive solution](https://github.com/MaxPowerWasTaken/discrete_opt_coursera/blob/6d8f8a854b1a6a13a5b22d6bf9997ed2d63ca49d/knapsack/solver.py#L7):

```python
def solve_it(input_data):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []

    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, int(parts[0]), int(parts[1])))

    # a trivial algorithm for filling the knapsack
    # it takes items in-order until the knapsack is full
    value = 0
    weight = 0
    taken = [0]*len(items)

    for item in items:
        if weight + item.weight <= capacity:
            taken[item.index] = 1
            value += item.value
            weight += item.weight
    
    # prepare the solution in the specified output format
    output_data = str(value) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, taken))
    return output_data
```


...there's a bit more scaffolding code as well, namely a `submit.py` script that calls this `solve_it` function on progressively larger input data files, sends the solutions to coursera's server and then records the scores.
``` ```   

...As a quick aside, I much prefer the user experience for the programming assignments in this course-- programming in my own editor/environment, and submitting my solutions via a script from my own terminal-- to the alternative that some MOOCs have, where all programming assignments are done in a jupyter notebook on their server.

Anyway, the naive solution starter code we were given scores 3/10 on each of the six problems, for 18/60 overall.

### Coding our first solution 

The optimization topics covered in this first week were:
- Greedy algorithms
- Dynamic Programming
- Relaxation, Branch and Bound

So for my first solution, as a baseline, I started with a simple value-dense greedy selection:

In [ ]:
import numpy as np
import pandas as pd

def unzip(iterable):
    ''' thanks to https://stackoverflow.com/a/22115957/1870832'''
    return zip(*iterable)


def parse_data(input_data):
    ''' Convert string input into set of np arrays, along with capacity'''

    lines = str.splitlines(input_data)

    # first line is n,K
    capacity = int(lines[0].split()[1])

    # convert list of 'value weight' strings to  unzipped lists of values, weights
    pairs = [x.split() for x in lines[1:]]  
    values, weights = unzip(pairs)         

    # convert from str to int
    values = [int(v) for v in values]
    weights = [int(w) for w in weights]

    return values, weights, capacity


def greedy_select(values, weights, K):
    ''' Value-dense greedy selection algorithm'''
    
    # convert to DF, add new column for value-density 
    df = pd.DataFrame({'value': values, 'weight': weights, 'capacity': K})
    df['value_density'] = df.value / df.weight

    # identify which items to select (top items by value-density, 
    # until cumulative-weight > capacity)
    df = df.sort_values(by="value_density", ascending=False)
    df['cumulative_weight'] = df.weight.cumsum()

    # return whether to select each item, **in original order given**, along with total value
    df['item_selected'] = np.where(df.cumulative_weight <= df.capacity, 1, 0)

    total_value = df.loc[df.item_selected==1, 'value'].sum()
    item_selections = list(df.sort_index().loc[:, 'item_selected'])

    return total_value, item_selections

def solve_it(input_data):
    
    # parse the input
    (values, weights, K) = parse_data(input_data)
    
    # value-dense greedy algorithm for filling the knapsack
    obj, item_selections = greedy_select(values, weights, K)

    # prepare the solution in the specified output format
    PROVED_OPTIMAL = 0
    output_data = f"{obj} {PROVED_OPTIMAL}\n"
    output_data += ' '.join(str(i) for i in item_selections)
    return output_data


If you want to see the diff of the original `solve_it` file to the value-dense greedy version above, [you can check it out here](https://github.com/MaxPowerWasTaken/discrete_opt_coursera/commit/9e9f4e87df79fd704b9ec6ee045e7ff0d7ce0f4c#diff-57214dafea7890eb954c73d3f58b3d17)

Alright so how does this solution do? We can check by running the `submit.py` script we're given as part of the assignment, which will run our `solve_it` code for six incrementally larger data files, send the solutions to the coursera server, where it will be scored:

```bash
(knapenv) maxepstein@pop-os:~/discrete_opt/knapsack$ python submit.py
==
== Knapsack Solution Submission 
==
Hello! These are the assignment parts that you can submit:
1) Knapsack Problem 1
2) Knapsack Problem 2
3) Knapsack Problem 3
4) Knapsack Problem 4
5) Knapsack Problem 5
6) Knapsack Problem 6
0) All
Please enter which part(s) you want to submit (0-6): 0
```
...As a quick aside, I much prefer the user experience for the programming assignments in this course-- programming in my own editor/environment, and submitting my solutions via a script from my own terminal-- to the alternative that some MOOCs have, where all programming assignments are done in a jupyter notebook on their server.

Anyway, back to the scoring. My value-dense greedy solution scores 3/10 on each of the six problems, for 18/60 overall. That's lower than I expected, which is actually encouraging; we'll have to do something reasonable to get a non-terrible score. Let's move on to a non-baseline solution.

### Second Approach: Dynamic Programming.
Dynamic Programming(DP) has the advantage of guaranteeing an optimal solution...if we have the compute time & RAM to complete the solution. 

DP is kinda like recursion, in that we use a set of `recurrence relations` (Bellman Equations) to define how we can break a given problem into smaller sub-problems. We also have a solution for the simplest, sorta atomic level of subproblem. However, recursion is top-down, in that you start with your 'real problem', break it into subproblems, break those subproblems into further subproblems, etc, all the way until you hit that atomic level that you have a solution for. DP, by contrast, is bottom up; it starts at the atomic level you have a pre-determined solution for, and uses the recurrence relations to keep building out a solution one level higher, then another level higher, until you have the answer for the level you need.

The recurrence relations for the DP solution of the Knapsack problem is as follows:

Let's define `O(k,j)` as the optimal value we can get for a knapsack of capacity `k` and with `j` items to consider. 

- $O(k,0) = 0$ for all $k$, because we get no value from no items.
- Now assume, while trying to solve for any $O(k,j)$, we had the solution to $O(k, j-1)$. Then we would just need to find the increase in our optimal value for adding that `j`th item...

The value of $O(k,j)$ as a function of $O(k,j-1)$ is:
 - $O(k,j) = O(k, j-1)$ if $w_j > k$ (if the j'th item doesn't fit in the knapsack, the optimal value is the same as for previous j-1 items)
 - $O(k,j) = max(O(k,j-1), vj + O(k-wj,j-1))$ otherwise (if $w_j \leq k$ 
    - this is less intuitive but see [this lecture](https://www.coursera.org/learn/discrete-optimization/lecture/wFFdN/knapsack-4-dynamic-programming) for more explanation

If all this seems complicated, I think DP/Bellman Equations are much easier to grok when the recurrence relations contain only a single variable; the canonical example seems to be using DP or recursion to find the n'th item in the fibonacci sequence, so if you're still confused google that for a gentler introduction to DP. The DP lecture link above from this Discrete Opt course ([also here](https://www.coursera.org/learn/discrete-optimization/lecture/wFFdN/knapsack-4-dynamic-programming)) is helpful, although I do think from about 7:00 on is much more intuitive and easier to follow than the first 7:00. So anyway if you follow that link, don't give up in the first 7:00!

Anyway, onto the code for the DP solution...

In [1]:
def dp_select(values, weights, capacity):
    """Use dynamic programming to build up to solution for all items"""
    
    # convert params to n,K notation as used in equations in dynamic programming notes
    n = len(values)
    K = capacity
    
    # calculate table of optimal value by j,k (j items in 0..n, k is all capacities in 0..K)
    # see 9:00 - 12:30 here: https://www.coursera.org/learn/discrete-optimization/lecture/wFFdN/knapsack-4-dynamic-programming
    values_table = np.zeros((K+1,n+1), dtype=np.uint32)
    
    print("building DP optimal-value table for n,K: ", n, ",", K)
    for j in tqdm(range(1, n+1)):
        item_weight = weights[j-1]
        item_value  = values[j-1]
        for k in range(1, K+1):
            if item_weight > k:
                values_table[k,j] = values_table[k, j-1]
            else:
                values_table[k,j] = max(values_table[k, j-1], item_value + values_table[k-item_weight, j-1])
    optimal_value = values_table[-1, -1]
    print(f"optimal value is {optimal_value}. Now proceeding to derive final item-set")

    # from this table of optimal values, we now need to derive final item-set for optimal solution
    # logic of code below explained 12:30 - 14:00 at https://www.coursera.org/learn/discrete-optimization/lecture/wFFdN/knapsack-4-dynamic-programming
    taken = [0] * len(values)
    k = K   # in keeping w/ eqs, K is total capacity but k is k'th row as we move through j,k table
    for j in range(n, 0, -1):
        if values_table[k,j] != values_table[k,j-1]:
            taken[j-1] = 1
            k = k - weights[j-1]
    
    return optimal_value, taken


def solve_it(input_data):
    
    # parse the input
    (values, weights, K) = parse_data(input_data)
    
    # value-dense greedy algorithm for filling the knapsack
    obj, item_selections = dp_select(values, weights, K)

    # prepare the solution in the specified output format
    PROVED_OPTIMAL = 1
    output_data = f"{obj} {PROVED_OPTIMAL}\n"
    output_data += ' '.join(str(i) for i in item_selections)
    return output_data


Again, if you'd rather see it as a diff from the previous code, [check here](------------------------)

So how'd the DP solution do?
- For Problem 1, with n=50 & K=100,000: Optimal Solution found in 3 seconds
- For Problem 2, with n=50 & K=341,045: Optimal Solution found in 32 seconds
- For Problem 3, with n=200 , 100000